# First model 

This model will try to predict wether pe is present or not on a single image (grey level)

## Imports

In [1]:
import numpy as np

In [2]:
import torch
torch.cuda.empty_cache()

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

PyTorch version: 1.7.1
GPU found :)


# Load datasets

In [3]:
channel0_dataset = torch.load('datasets/channel0.pt')
channel1_dataset = torch.load('datasets/channel1.pt')
channel2_dataset = torch.load('datasets/channel2.pt')

## Simple CNN from scratch

In [4]:
class ModelV1(nn.Module):
    
    def __init__(self, input_channels):
        """Simple Convnet with 2 convolution layer + pooling, with 2 fully connected at the end"""
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32*62*62 , 120)
        self.fc2 = nn.Linear(120, 2)
        
        
    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32*62*62) # Flatten
        x = F.relu(self.fc1(x))
        
        # Binary classification
        out = torch.sigmoid(self.fc2(x))
        
        return out

## Split data into train and test

In [5]:
dataset = channel0_dataset

# Compute sizes for the split
test_ratio = 0.2
test_size = int(test_ratio * len(dataset))
train_size = len(dataset) - test_size

# Get indices for train and test, then sample them
indices = list(range(len(dataset)))
np.random.seed(12345)
np.random.shuffle(indices)

train_indices, test_indices = indices[train_size:], indices[:test_size]

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Data loaders for the split
train_loader = DataLoader(dataset, batch_size=1, sampler=train_sampler)
test_loader = DataLoader(dataset, sampler=test_sampler)

## Model Training

In [6]:
num_epoch = 5
step_count = len(train_loader)
loss_function = nn.CrossEntropyLoss()
model = ModelV1(input_channels=1).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(num_epoch):
    for i, sample in enumerate(train_loader):
        
        image = sample['image'].to(device, dtype=torch.float)
        target = sample['target'].to(device)
        
        # Reset gradiant
        optimizer.zero_grad()
        
        # Forward pass
        pred = model(image)
        
        # Compute loss
        loss = loss_function(pred, target)
        
        # Backprop
        loss.backward()
        optimizer.step()
        
        # Debug
        if((i+1) % 100 == 0):
            print(
                        f"Epoch [{epoch + 1}/{num_epoch}]"
                        f", step [{i + 1}/{step_count}]"
                        f", loss: {loss.item():.4f}"
                    )

Epoch [1/5], step [100/1254], loss: 0.6923
Epoch [1/5], step [200/1254], loss: 0.6908
Epoch [1/5], step [300/1254], loss: 0.7001
Epoch [1/5], step [400/1254], loss: 0.7042
Epoch [1/5], step [500/1254], loss: 0.6876
Epoch [1/5], step [600/1254], loss: 0.6957
Epoch [1/5], step [700/1254], loss: 0.6909
Epoch [1/5], step [800/1254], loss: 0.6973
Epoch [1/5], step [900/1254], loss: 0.6891
Epoch [1/5], step [1000/1254], loss: 0.6909
Epoch [1/5], step [1100/1254], loss: 0.6866
Epoch [1/5], step [1200/1254], loss: 0.6889
Epoch [2/5], step [100/1254], loss: 0.6894
Epoch [2/5], step [200/1254], loss: 0.6950
Epoch [2/5], step [300/1254], loss: 0.6957
Epoch [2/5], step [400/1254], loss: 0.6998
Epoch [2/5], step [500/1254], loss: 0.6966
Epoch [2/5], step [600/1254], loss: 0.6958
Epoch [2/5], step [700/1254], loss: 0.7000
Epoch [2/5], step [800/1254], loss: 0.6749
Epoch [2/5], step [900/1254], loss: 0.7219
Epoch [2/5], step [1000/1254], loss: 0.7091
Epoch [2/5], step [1100/1254], loss: 0.6959
Epoch 

In [7]:
# Save model 
torch.save(model.state_dict(), './models/model_v1.h5')

## Test the model

In [8]:
# Load model for evaluation
trained_model = ModelV1(input_channels=1)
trained_model.load_state_dict(torch.load('./models/model_v1.h5'))
trained_model.to(device)

ModelV1(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=123008, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=2, bias=True)
)

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for sample in test_loader:
        
        image = sample['image'].to(device, dtype=torch.float)
        target = sample['target'].to(device)
        
        output = model(image)
        prob, index = torch.max(output, 1)
        total += target.size(0)
        correct += (index == target).sum().item()

print(f'Accuracy of the network on the {len(test_loader)} test images: {100 * correct / total}')

Accuracy of the network on the 1254 test images: 50.87719298245614
